# ABS Monthly Consumer Price Index Indicator 6484

Caution - the month;y indicator is problematic, and easily misunderstood
 * The CPI indicator is not the whole basket of goods found in the CPI
 * Some included items are only updated quarterly or annually - these items overly inflate the months in which they appear, 
   and do no contribute anything in the months they do not appear.
 * Some volatile items (especially travel) are included in the series that excluded volatile items.
 * The monthly series is way more noisy than the quarterly CPI series.
 * Annualising less than 12 month changes is problematic, and should be avoided.
 * Strongly advised: Focus on the annual trimmed mean indicator. Just ignore month-to-month changes in all series. 
   Favour the seasonally adjusted headline series over the original headline series. 

## Python set-up

In [1]:
# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol as mc

# local imports
from plotting import (
    calc_growth,
    finalise_plot,
    line_plot,
    plot_growth,
)
from abs_helper import get_abs_data, ANNUAL_CPI_TARGET_RANGE, MONTHLY_CPI_TARGET

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, recent = get_abs_data("6484.0")

## Plot

### Plot growth - combined line/bar charts

In [3]:
def cpi_growth():
    """Produce CPI growth charts."""

    rows = meta[meta[mc.did].str.contains("Index Numbers")]

    for index, row in rows.iterrows():
        # get data for the plot
        did, table = row[mc.did], row[mc.table]
        series_type, series_id = row[mc.stype], row[mc.id]
        name = did.split(";")[1].strip()
        index = abs_dict[table][series_id]
        growth = calc_growth(index)

        # plot the data
        very_recent = (pd.Timestamp.today() - pd.DateOffset(months=18)).strftime(
            "%Y-%m-%d"
        )
        for period in [recent, very_recent]:
            ax = plot_growth(
                *growth, from_=period, annotate=9 if period == very_recent else False
            )
            if ax:
                long_name = 50
                separator = " " if len(name) < long_name else "\n"
                finalise_plot(
                    ax,
                    title=f"Price Growth:{separator}{name}",
                    ylabel="Per cent",
                    rfooter=source,
                    axhspan=ANNUAL_CPI_TARGET_RANGE,
                    axhline=MONTHLY_CPI_TARGET,
                    legend={"loc": "best", "fontsize": 8},
                    lfooter=f"Australia: {series_type.capitalize()} series. ",
                    tag=period,
                    y0=True,
                    zero_y=True,
                    show=SHOW,
                )


cpi_growth()

### Selected sub items

In [4]:
def food_fuel() -> None:
    """Produce food and fuel sub-item charts."""

    annual = "Corresponding Month of Previous Year"
    selectors = {
        "Food and non-alcoholic beverages": {
            "Food and non-alcoholic beverages": mc.did,
            "Original": mc.stype,
            annual: mc.did,
        },
        "Automotive fuel": {
            "Automotive fuel": mc.did,
            "Original": mc.stype,
            annual: mc.did,
        },
    }

    data = pd.DataFrame()
    for name, selector in selectors.items():
        found = ra.search_meta(
            meta, selector, verbose=False, validate_unique=True
        ).iloc[0]
        table, series_id = found[mc.table], found[mc.id]
        data[name] = abs_dict[table][series_id]

    title = "Monthly CPI Indicator - Annual Growth - Food & Fuel"
    tag = "1"
    series_type = "Original series"
    line_plot(
        data,
        title=title,
        ylabel="Per cent",
        width=(1, 2),
        tags=tag,
        legend={"loc": "best", "fontsize": "x-small"},
        rfooter=f"{source}",
        lfooter=f"Australia: {series_type}",
        y0=True,
        axhspan=ANNUAL_CPI_TARGET_RANGE,
        show=SHOW,
    )


food_fuel()

In [5]:
def housing() -> None:
    """Produce housing related charts."""

    annual = "Corresponding Month of Previous Year"
    selectors = {
        "Rents": {
            "Rents": mc.did,
            "Original": mc.stype,
            annual: mc.did,
        },
        "New dwelling purchase by owner-occupiers": {
            "New dwelling purchase by owner-occupiers": mc.did,
            "Original": mc.stype,
            annual: mc.did,
        },
    }

    data = pd.DataFrame()
    for name, selector in selectors.items():
        found = ra.search_meta(
            meta, selector, verbose=False, validate_unique=True
        ).iloc[0]
        table, series_id = found[mc.table], found[mc.id]
        data[name] = abs_dict[table][series_id]

    title = "Monthly CPI Indicator - Annual Growth - Residential"
    tag = "2"
    series_type = "Original series"
    line_plot(
        data,
        title=title,
        ylabel="Per cent",
        width=(1, 2),
        tags=tag,
        legend={"loc": "best", "fontsize": "x-small"},
        rfooter=f"{source}",
        lfooter=f"Australia: {series_type}",
        y0=True,
        axhspan=ANNUAL_CPI_TARGET_RANGE,
        show=SHOW,
    )


housing()

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Jul 03 2024 07:42:05

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

readabs: 0.0.5
pandas : 2.2.2

Watermark: 2.4.3



In [7]:
print("Done")

Done
